#**CHAPTER 1.PERSONAL FINANCE TRIAGE**
---

##REFERENCE

https://chatgpt.com/share/6995b4e7-d054-8012-99bb-5d6710795567

##0.CONTEXT

This first Colab notebook is intentionally designed as a pedagogical “minimum viable agent system” for finance. The objective is not to deliver financial advice, and it is not to impress the reader with complexity for its own sake. The objective is to teach the student what it means to build an agentic workflow as an explicit system: a controlled process with state, roles, routing, and termination. The motivating domain is personal finance because it is the most natural setting in which the limitations of naive prompting become obvious. Personal finance requests arrive incomplete, ambiguous, and underspecified. A user might provide income but not expenses, savings but not liabilities, goals but not horizons. In professional settings, the first step is never “answer the question.” The first step is triage: determine what you know, determine what you do not know, and decide what must happen next. Notebook 1 turns that commonplace professional pattern into a concrete, runnable architecture.

The pedagogical value for the user is threefold. First, the notebook forces architectural thinking. Instead of writing one prompt that tries to do everything, the workflow is decomposed into roles that are easy to reason about, easy to debug, and easy to modify. Second, it forces explicitness. The system stores information in a structured state object, and decisions are driven by that state rather than by hidden conversational momentum. Third, it forces control. The system includes a retry loop with a strict bound. That may sound like a small detail, but it introduces an essential discipline: real agent systems must have termination conditions. Without termination, “autonomy” becomes unbounded execution, which is not intelligence but instability.

A key principle in this notebook is that agents are not magical. Agents are modular components embedded inside a graph. A model call is not an architecture. An architecture is a pattern of dependencies: what runs when, under what conditions, and with what shared context. LangGraph is used precisely because it lets us represent that architecture explicitly. The student is learning one tool and one abstraction. The point is not to explore an ecosystem of frameworks. The point is to internalize the graph mental model so deeply that new architectures become simple variations of the same underlying primitives.

The notebook also makes a strong claim about what matters in the classroom: speed and clarity. The strict use of **claude-haiku-4-5-20251001** is not a technical footnote; it is an instructional choice. If the model is slow, the attention of the room collapses. If the feedback loop is delayed, students stop interacting with the architecture and start waiting for outputs. For pedagogy, responsiveness often matters more than maximal reasoning depth. This notebook is designed to be run live, modified live, and debugged live. It is therefore grounded in the principle that we teach architecture by experimentation, not by proclamation.

The visualization requirement is not optional. The architecture must be seen. The Mermaid graph is the first-class learning artifact, and the code is secondary. A student should be able to look at the diagram and explain the system: what the nodes represent, what the conditions represent, where loops occur, and where the system terminates. The notebook includes a programmatic Mermaid renderer because visualization is part of correctness. If the graph does not render, the pedagogical objective fails. This is also a subtle but important lesson in professional engineering practice: explanations and diagrams are not decoration. They are instruments of coordination and review. In real teams, the architecture is what gets debated first.

From the student’s perspective, Notebook 1 should leave behind a durable mental model. The mental model is that an agentic system is not a single prompt. It is a controlled process composed of three essential elements: **state**, **agents**, and a **graph**. If the student truly learns these definitions, the rest of the series becomes a matter of combining and extending them.

What follows is a definition-driven description of the pipeline itself, written to make those primitives precise.

**The pipeline in this notebook is a graph-based triage loop.** It begins with a user message and ends with a structured representation of the information plus a set of follow-up questions when the information is incomplete. The pipeline is not “linear,” even though it looks simple, because the next step is chosen conditionally. The intake node can route forward or route into a loop depending on what the system discovers. That conditional control is exactly what makes it a graph, not a chain.

To understand the pipeline, we start with definitions.

**A state is a structured representation of the system’s current knowledge and progress.** In this notebook, state is implemented as a typed dictionary with explicitly named fields. The most important pedagogical point is that state is not the conversation. State is the system’s own internal data structure that persists across nodes. It is inspectable and modifiable. It is the system’s memory in the most literal sense: a collection of variables that can be read, updated, and tested. In the notebook, the state includes fields such as the original user input, accumulating working notes, a boolean flag indicating whether information is complete, a retry counter, and a place to store final output. These fields are not “nice to have.” They are the mechanism through which the architecture becomes controllable.

State serves three functions. First, it records evidence: what the user said and what agents have produced so far. Second, it enables decision-making: routing depends on `info_complete` and `retry_count`. Third, it enables debugging: if a run produces unexpected behavior, the student can inspect the state to see which node wrote what and why the router chose the path it chose. In a pure chat setting, those internal choices are opaque. In a graph setting, they are explicit.

A second definition is equally important.

**An agent is a role-defined transformation that reads state and writes state.** In this notebook, an agent is not an “entity” with autonomy. It is a node function with a name, a prompt (system instruction), and a predictable contract. The contract is simple: given the current state, the agent produces an output and appends it to working notes. Optionally, a post-processing hook updates specific fields such as `info_complete` or `retry_count`. This is the pedagogical heart of modularity. The student learns that an agent should have a narrow responsibility and a clear effect on state.

The notebook contains three agents.

The **Intake agent** is responsible for diagnosis. It reads the current working notes (which include user input and any prior outputs) and determines whether essential information is missing. Its output is not primarily advice. Its output is a classification expressed in a way the system can use. In this notebook, the intake agent is instructed to explicitly write the word “MISSING” when essential information is absent. That design is intentionally simple and robust for teaching: it creates a clear signal for downstream routing. The post-processing hook for intake scans the output and sets `info_complete` based on whether “MISSING” appears. In more advanced notebooks, this heuristic will evolve into structured schemas and validators, but the pedagogical idea is the same: an agent produces an output, and the system extracts a decision signal from that output to drive control flow.

The **Clarification agent** is responsible for targeted questioning. It only runs when intake indicates incompleteness. It asks concise follow-up questions designed to fill the gaps. It also increments the retry counter through a post-processing hook. This teaches another essential idea: an agent node can be used to modify state even when the user is not actively responding in the loop. The retry counter exists to prevent endless cycling. It turns the loop into a bounded procedure. Again, the details are simple because the principle is the goal.

The **Structuring agent** is responsible for organization. Once information is sufficiently complete, this agent converts the accumulated notes into a structured representation: income, expenses, assets, liabilities, and goals. This does not require optimization. It is a transformation from unstructured to structured. Pedagogically, this is important because students learn that many valuable applications of LLMs in finance are not “predictive” tasks but “structuring” tasks: turning messy inputs into clean, reviewable representations.

The third foundational definition is:

**A graph is an explicit model of control flow: a set of nodes connected by edges that define what can happen next.** In LangGraph, a graph is constructed by adding nodes (agent functions) and edges (transitions) and then compiling the result into an executable object. The key property of a graph is that the next step is not necessarily fixed. The pipeline is a graph because it contains conditional edges and a cycle. When the intake node completes, the system does not simply proceed to the next node. Instead, a routing function reads the state and chooses the next node: clarify or structure. This choice is not made by the model implicitly; it is made by the architecture explicitly.

The graph in Notebook 1 has one entry point and one terminal path, but it also has a loop. The entry point is intake. After intake, the routing function examines `info_complete` and `retry_count`. If information is incomplete and the retry limit has not been reached, the graph routes to clarification. Clarification then routes back to intake. If information is complete, or if the retry limit has been reached, the graph routes to structuring and then terminates. This is the simplest possible demonstration of a nontrivial graph. It already contains the core ingredients that make agentic architectures powerful: feedback, conditionality, and termination.

This definition-driven framing is the essence of what the pipeline achieves for the user. It turns the concept of “agents” into something that can be built, inspected, and understood as a system. The student learns that good agent design begins not with clever prompts but with clear state, clear roles, and clear routing logic. The student also learns that a diagram is not optional. The diagram is the mental model made visible. If you can narrate the diagram, you understand the system. If you cannot narrate the diagram, you are still guessing.

In later notebooks, we will expand the architecture in disciplined steps: multiple termination paths, hard branching vetoes, parallel committees, tool-augmented nodes, retrieval routers, and supervised multi-agent constellations. But the student should recognize that all of those expansions are variations on what is learned here. The primitives remain state, agents, and graphs. The system becomes more complex only by adding nodes, expanding state, and enriching the topology. That is the point of starting here. Notebook 1 is not boring when understood correctly. It is the first place where the student stops thinking of AI as a chat interface and starts thinking of AI as an engineered workflow.

This is why the notebook is worth the time. It gives the student a stable conceptual map from problem to architecture to implementation. It shows that practical agentic systems are not mystical. They are explicit. They are composable. They are debuggable. And they are teachable.


##1.LIBRARIES AND ENVIRONMENT



The first cell establishes the operational environment for the notebook and signals an important pedagogical commitment: we are building a system, not writing a script. Installing the required libraries is not just a technical step; it makes explicit which tools define the conceptual boundary of the exercise. The student sees immediately that the system rests on three foundations: **LangGraph** for orchestration, **Anthropic’s Claude 4.5** for reasoning, and core Python typing for structured state. This is deliberate. The goal is to minimize conceptual sprawl. One orchestration tool, one model family, one typed state representation.

This cell also reinforces the idea that an agentic architecture is not abstract theory. It is executable. When students run this cell and see the environment initialize, they understand that what follows is not metaphor. It is code that governs behavior. This matters for pedagogy because agent discussions often remain at the conceptual level. By starting with concrete execution, we anchor the lesson in practice.

Another subtle but important objective of this cell is psychological. When the notebook prints confirmation messages, it provides feedback. Fast feedback builds trust in the system. In a classroom setting, responsiveness preserves attention. The fact that the model choice is strict and pinned to a specific identifier is also an instructional statement: architectural stability is more important than chasing the latest release. Students should learn that reproducibility and clarity outweigh novelty.

Finally, this cell signals modularity. By isolating installation and imports, we make the computational boundary visible. Everything that follows will operate inside this boundary. The student begins to internalize that an agent system is assembled from components. Each component must be named, imported, and understood. The architecture does not emerge spontaneously. It is constructed intentionally from defined parts.


In [5]:
# CELL 1 — Install + Imports + Boot
!pip install -q langgraph anthropic pydantic

import os, json, uuid, datetime
from typing import TypedDict
from IPython.display import display, HTML

from anthropic import Anthropic
from langgraph.graph import StateGraph, END

print("Booting Notebook #1 — LangGraph Retry Triage (Colab)")


Booting Notebook #1 — LangGraph Retry Triage (Colab)


##2.MODEL

###2.1.OVERVIEW



The second cell formalizes model configuration and secret management. At first glance, this may appear procedural, but pedagogically it introduces a critical architectural distinction: the separation between **cognitive engine** and **system structure**. The model, defined here as claude-haiku-4-5-20251001, is the reasoning substrate. It is not the architecture itself. By isolating model configuration in its own cell, we teach that the model can be swapped without rewriting the graph.

Using Colab’s userdata retrieval is equally instructive. It demonstrates secure parameterization. Students learn that credentials are not hard-coded; they are injected at runtime. This mirrors professional engineering practices where secrets are managed externally. Even though this notebook is pedagogical rather than production-grade, it quietly instills discipline.

The explicit printing of model name, run ID, and timestamp reinforces reproducibility. Every run is uniquely identifiable. This transforms the notebook from a transient demonstration into a traceable experiment. Students should understand that an agent system is not just something that “runs.” It produces artifacts tied to a specific configuration.

Another pedagogical insight lies in the model selection itself. Haiku 4.5 is chosen for speed. This reinforces the lesson that architectural reasoning does not require maximal model complexity. Students must grasp that the architecture governs the workflow. The model executes instructions within that workflow. If the architecture is clear, the model can be lightweight.

This cell therefore teaches configuration discipline, separation of concerns, secure key management, and reproducibility. It grounds the student in the idea that intelligent systems are parameterized systems. Control over configuration is the first layer of control over behavior.


###2.2.CODE AND IMPLEMENTATION

In [15]:
# CELL 2 — Model (STRICT) + Colab Secrets API Key (unchanged logic, just keep canonical)
MODEL = "claude-haiku-4-5-20251001"

api_key = None
try:
    from google.colab import userdata
    api_key = userdata.get("ANTHROPIC_API_KEY")  # <-- your secret name (case-sensitive)
    print("Colab userdata available.")
except Exception:
    print("google.colab.userdata not available (not running in Colab?).")

if not api_key:
    api_key = os.environ.get("Anthropic_API_KEY")

if not api_key:
    raise RuntimeError(
        "Anthropic API key not found. Set Colab Secret Anthropic_API_KEY or env var Anthropic_API_KEY."
    )

client = Anthropic(api_key=api_key)

RUN_ID = str(uuid.uuid4())
TIMESTAMP = datetime.datetime.now(datetime.timezone.utc).isoformat()

print("Model:", MODEL)
print("Run ID:", RUN_ID)
print("Timestamp (UTC):", TIMESTAMP)


Colab userdata available.
Model: claude-haiku-4-5-20251001
Run ID: f6aa292e-d90f-46b2-b484-8a05ccf0434d
Timestamp (UTC): 2026-02-18T13:17:06.720723+00:00


##3.STATE DEFINITION


###3.1.OVERVIEW


Cell three defines the **state**, and this is arguably the most important conceptual pivot in the notebook. In conversational AI, context is implicit. In an agent graph, context becomes explicit data. The state object is the system’s working memory. It is a structured container that persists across nodes and drives routing decisions.

Defining state as a TypedDict is pedagogically powerful. It signals that the architecture operates over named fields rather than unstructured text streams. Each field has semantic meaning: user_input stores original data, working_notes accumulates transformations, info_complete governs routing, retry_count bounds iteration, and final_output represents terminal structure. Students learn that system memory is not magical. It is engineered.

The presence of info_complete and retry_count demonstrates that state is not merely descriptive; it is operational. The router will read these fields to determine the next step. This reinforces a central principle: control flow depends on data, not on prompt speculation. When students inspect the state object, they can see exactly why the system chose a particular path.

The typed schema also promotes clarity and debugging. If behavior is unexpected, one can inspect state at any stage. This is how professional systems are debugged. Not by guessing what the model “intended,” but by examining the data driving transitions.

Pedagogically, this cell marks the shift from thinking of AI as conversation to thinking of AI as computation. The student begins to internalize that an agentic workflow is a function over state. Without structured state, there is no architecture—only sequential prompts.


###3.2.CODE AND IMPLEMENTATION

In [16]:
# CELL 3 — State Definition
class AgentState(TypedDict):
    user_input: str
    working_notes: str
    info_complete: bool
    retry_count: int
    final_output: str

print("State schema ready:", AgentState.__annotations__)


State schema ready: {'user_input': <class 'str'>, 'working_notes': <class 'str'>, 'info_complete': <class 'bool'>, 'retry_count': <class 'int'>, 'final_output': <class 'str'>}


##4.CLAUDE WRAPPER

###4.1.OVERVIEW



This cell introduces two essential abstractions: the Claude call wrapper and the AgentNode constructor. Together they define what an **agent** is in this system. An agent is not an autonomous being. It is a role-defined transformation that reads state and writes state.

The claude_call function encapsulates the interaction with the model. This separation isolates cognition from orchestration. If we later changed models, token limits, or temperature settings, we would modify this wrapper without touching the graph logic. Students learn that reasoning engines should be abstracted from workflow logic.

The AgentNode constructor is even more instructive. It creates a closure that accepts state, calls the model, appends output to working_notes, and optionally executes a post-processing hook. This pattern teaches modularity. Each agent has a name, a prompt, and a defined effect on state. Agents become interchangeable components.

The post-processing hook is particularly pedagogical. It shows that model outputs can influence structured variables. For example, detecting the word MISSING updates info_complete. This demonstrates that architecture can extract signals from language and convert them into machine-readable control data. Students see how unstructured text can drive structured decision-making.

By encapsulating this logic, we reduce repetition and enforce a clean mental model. Agents are standardized units. They read state, produce output, update state, and return control. This cell teaches abstraction, modular design, and separation of concerns—core principles that will scale throughout the notebook series.


###4.2.CODE AND IMPLEMENTATION

In [17]:
# CELL 4 — Claude Wrapper + AgentNode Abstraction
def claude_call(system: str, user: str, max_tokens: int = 500) -> str:
    response = client.messages.create(
        model=MODEL,
        max_tokens=max_tokens,
        system=system,
        messages=[{"role": "user", "content": user}],
    )
    return response.content[0].text


def AgentNode(name: str, system_prompt: str, post_process=None):
    def node(state: AgentState):
        output = claude_call(system_prompt, state["working_notes"])
        state["working_notes"] += f"\n\n[{name}]:\n{output}"
        if post_process:
            post_process(state, output)
        return state

    return node

print("Agent abstraction ready.")


Agent abstraction ready.


##5.STATE MUTATION

###5.1.OVERVIEW

CELL 5 EXPLANATION

Cell five defines the post-processing hooks that mutate state based on agent output. This is where the architecture begins to show its operational character. The intake_post function inspects model output to determine whether essential information is missing. The clarify_post function increments the retry counter.

The pedagogical value here lies in demonstrating that agents can have side effects beyond textual output. A naive implementation would simply append responses and move on. Instead, this notebook extracts a structured signal from the output and stores it in state. Students learn that intelligent systems often rely on hybrid logic: language generation plus deterministic evaluation.

The use of a simple heuristic—checking for the word MISSING—is intentional. The point is not sophistication; the point is clarity. Students can immediately understand why the system sets info_complete to true or false. In later notebooks, this mechanism may evolve into schema validation or structured parsing, but the conceptual scaffold remains the same.

The retry counter reinforces bounded loops. Every iterative process must have a limit. By incrementing retry_count explicitly, the system guards against infinite cycles. This introduces the principle of termination control, a foundational requirement in real agent systems.

This cell teaches that architecture includes not only nodes and edges but also controlled state mutation. The student sees that agentic workflows are a blend of probabilistic reasoning and deterministic governance. This interplay is central to scalable AI systems.


###5.2.CODE AND IMPLEMENTATION

In [18]:
# CELL 5 — Post-processing Hooks (State Mutation)
def intake_post(state: AgentState, output: str):
    # Minimal completeness heuristic for demo: if agent says "MISSING", route to clarify
    state["info_complete"] = ("MISSING" not in output.upper())

def clarify_post(state: AgentState, output: str):
    state["retry_count"] += 1

print("Post-processing hooks configured.")


Post-processing hooks configured.


##6.GRAPH CONSTRUCTION

###6.1.OVERVIEW

CELL 6 EXPLANATION

This cell constructs the LangGraph itself. Here the abstract definitions of state and agents become an executable control structure. Nodes are added, an entry point is defined, conditional edges are registered, and the graph is compiled.

The most pedagogically important feature is the router function. It inspects state and decides which node should execute next. This transforms the system from a chain into a graph. The next step is not predetermined; it depends on data. Students see clearly that control flow is programmable.

The conditional edges encode the branching logic. If information is incomplete and retry_count is below the threshold, the graph routes to clarification. Otherwise, it routes to structuring. This introduces the concept of a bounded feedback loop. Clarification feeds back into intake, forming a cycle. Structuring routes to END, forming termination.

Compiling the graph produces an executable artifact. Students should recognize that they have not merely written a sequence of calls but constructed a state machine. The architecture now has an identity independent of any single run.

This cell teaches the essence of LangGraph: explicit nodes, explicit transitions, and explicit control logic. It reinforces that architectural clarity is achieved by modeling flow declaratively rather than implicitly.


###6.2.CODE AND IMPLEMENTATION

In [19]:
# CELL 6 — LangGraph Construction with Conditional Retry Loop
builder = StateGraph(AgentState)

builder.add_node("intake", AgentNode(
    "Intake",
    "Analyze the financial input. If essential information is missing, explicitly write the word MISSING.",
    intake_post
))

builder.add_node("clarify", AgentNode(
    "Clarify",
    "Ask concise follow-up questions to complete missing financial data.",
    clarify_post
))

builder.add_node("structure", AgentNode(
    "Structure",
    "Organize the financial information into categories: income, expenses, assets, liabilities, goals."
))

def router(state: AgentState):
    # Retry at most 2 times (i.e., allow clarify->intake loop)
    if (not state["info_complete"]) and (state["retry_count"] < 2):
        return "clarify"
    return "structure"

builder.set_entry_point("intake")

builder.add_conditional_edges(
    "intake",
    router,
    {"clarify": "clarify", "structure": "structure"}
)

builder.add_edge("clarify", "intake")
builder.add_edge("structure", END)

graph = builder.compile()

print("LangGraph compiled with retry loop.")


LangGraph compiled with retry loop.


##7.VISUALIZATION

###7.1.OVERVIEW



The seventh cell renders the Mermaid diagram programmatically. This is not aesthetic embellishment; it is architectural exposition. The diagram is the conceptual twin of the graph defined in code. Students should compare the visual topology with the routing logic and see that they match.

Visualization reinforces comprehension. Many learners understand structure more quickly when they see flow arrows and conditional branches. The loop from clarification back to intake becomes immediately apparent. The decision node labeled Info Complete? makes routing logic tangible.

Rendering Mermaid via HTML injection also teaches a subtle lesson about engineering resilience. If visualization fails, architecture becomes opaque. Including rendering logic in code emphasizes that explanation is part of system design. In professional contexts, architecture diagrams are not optional—they are artifacts that support collaboration.

This cell therefore bridges implementation and pedagogy. It turns abstract flow into a shared visual language. Students should be encouraged to narrate the diagram aloud: where does it start, what happens if information is missing, when does it terminate? That narration indicates true understanding.

Ultimately, this cell reinforces that agentic systems are structured processes. The diagram externalizes that structure. Seeing the graph builds intuition that will support increasingly complex topologies in future notebooks.


###7.2.CODE AND IMPLEMENTATION

In [20]:
# CELL 7 — Mermaid Visualization (REWRITTEN: hardened Colab-compatible renderer + LangGraph diagram)
from IPython.display import HTML, display
import hashlib

def render_mermaid_locally(mermaid_code: str, *, theme: str = "default", mermaid_version: str = "10.6.1"):
    h = hashlib.md5(mermaid_code.encode("utf-8")).hexdigest()[:10]
    diagram_id = f"mermaid-diagram-{h}"

    html_template = f"""
<div style="border:1px solid rgba(0,0,0,0.08); border-radius:10px; padding:12px; margin:8px 0;">
  <div style="font-family: ui-monospace, SFMono-Regular, Menlo, Monaco, Consolas, 'Liberation Mono', 'Courier New', monospace;
              font-size: 12px; opacity: 0.8; margin-bottom: 8px;">
    Mermaid render: id={diagram_id} | v{mermaid_version}
  </div>
  <pre class="mermaid" id="{diagram_id}">{mermaid_code}</pre>
</div>

<script type="module">
  import mermaid from 'https://cdn.jsdelivr.net/npm/mermaid@{mermaid_version}/dist/mermaid.esm.min.mjs';

  mermaid.initialize({{
    startOnLoad: false,
    theme: '{theme}',
    securityLevel: 'loose',
    flowchart: {{
      useMaxWidth: false,
      htmlLabels: true
    }}
  }});

  const el = document.getElementById("{diagram_id}");
  if (el) {{
    await mermaid.run({{ nodes: [el] }});
  }}
</script>
"""
    display(HTML(html_template))


def display_langgraph_mermaid(graph, *, xray: int = 1, theme: str = "default", mermaid_version: str = "10.6.1"):
    mermaid_code = graph.get_graph(xray=xray).draw_mermaid()
    render_mermaid_locally(mermaid_code, theme=theme, mermaid_version=mermaid_version)

# Render the actual LangGraph diagram
display_langgraph_mermaid(graph, xray=1, theme="default", mermaid_version="10.6.1")


##8.EXECUTION

###8.1.OVERVIEW

CELL 8 EXPLANATION

Cell eight executes the graph with a deliberately incomplete example input. This is where theory meets behavior. The initial state is populated with minimal information to ensure that the intake agent flags missing essentials. When graph.invoke runs, the system enters the retry loop.

The pedagogical objective is experiential learning. Students observe retry_count incrementing and see clarification output appended to working_notes. The system’s loop is not hypothetical—it is observable. This concrete execution reinforces the idea that routing decisions are driven by state variables, not hidden model magic.

Running the graph also demonstrates compositional accumulation. working_notes grows with each agent invocation. Students see that state evolves over time, preserving history. This clarifies the difference between ephemeral chat responses and structured process memory.

Importantly, this execution cell invites experimentation. Students can modify the input and observe changes in routing. If the input includes complete financial details, the loop may not trigger. This interactive quality strengthens conceptual understanding.

By watching the graph operate, students internalize that an agent system is a controlled experiment. Each run produces traceable transitions. Execution is the moment where architectural design reveals its consequences.


###8.2.CODE AND IMPLEMENTATION

In [22]:
# CELL 8 — Example Execution (Should trigger the retry loop)
initial_state: AgentState = {
    "user_input": "I earn 80k per year and have 10k savings.",
    "working_notes": "USER INPUT:\nI earn 80k per year and have 10k savings.",
    "info_complete": False,
    "retry_count": 0,
    "final_output": ""
}

print("Invoking graph...\n")
result = graph.invoke(initial_state)

print("Retry count:", result["retry_count"])
print("\n========== FINAL WORKING NOTES ==========\n")
print(result["working_notes"])

Invoking graph...

Retry count: 2

========== FINAL WORKING NOTES ==========

USER INPUT:
I earn 80k per year and have 10k savings.

[Intake]:
# Financial Analysis

## Income
- **Annual Income:** $80,000
- **Monthly Income (gross):** ~$6,667

## Savings
- **Current Savings:** $10,000
- **Emergency Fund Status:** Limited (typically 3-6 months of expenses recommended)

## Missing Information

To provide a more comprehensive financial analysis, I would need:

- **Monthly expenses** - Housing, utilities, food, transportation, etc.
- **Debt** - Student loans, credit cards, car loans, mortgage
- **Employment stability** - Full-time/part-time, job security
- **Age and retirement timeline** - To assess long-term planning needs
- **Financial goals** - Homeownership, investments, education, travel
- **Dependents** - Family obligations affecting financial needs
- **Benefits** - Health insurance, 401(k) matching, pension
- **Other income sources** - Side hustles, investments, passive income

## Qu

##9.DIAGNOSTICS

###9.1.0VERVIEW


The ninth cell performs diagnostics and reflective inspection. It prints state variables, retry counts, and output lengths. This is not redundancy; it is reinforcement of observability. Students must learn that agent systems are inspectable systems.

Diagnostics reveal whether the routing logic behaved as intended. If info_complete is false and retry_count increased, the loop functioned correctly. If retry_count exceeds its bound, termination logic failed. Students see that debugging is an examination of state transitions.

Previewing working_notes reinforces the accumulation principle. Each node contributes to the evolving record. Observing this accumulation helps students grasp how context persists across iterations.

This cell teaches reflective engineering. Instead of assuming correctness, we verify it. Observability is a defining feature of scalable systems. In larger architectures, diagnostic layers become even more important.

Pedagogically, this cell reinforces a culture of inquiry. Students should ask: why did the router choose that path? What state triggered it? Diagnostics answer those questions. Understanding emerges not from belief but from inspection.


###9.2.CODE AND IMPLEMENTATION

In [25]:
# CELL 9 — Diagnostics / Reflection Outputs
print("Diagnostics")
print("Information complete:", result["info_complete"])
print("Retries performed:", result["retry_count"])
print("Working notes length:", len(result["working_notes"]))

# quick sanity: show first 400 chars
print("\nPreview (first 400 chars):\n", result["working_notes"][:2000])


Diagnostics
Information complete: False
Retries performed: 2
Working notes length: 7276

Preview (first 400 chars):
 USER INPUT:
I earn 80k per year and have 10k savings.

[Intake]:
# Financial Analysis

## Income
- **Annual Income:** $80,000
- **Monthly Income (gross):** ~$6,667

## Savings
- **Current Savings:** $10,000
- **Emergency Fund Status:** Limited (typically 3-6 months of expenses recommended)

## Missing Information

To provide a more comprehensive financial analysis, I would need:

- **Monthly expenses** - Housing, utilities, food, transportation, etc.
- **Debt** - Student loans, credit cards, car loans, mortgage
- **Employment stability** - Full-time/part-time, job security
- **Age and retirement timeline** - To assess long-term planning needs
- **Financial goals** - Homeownership, investments, education, travel
- **Dependents** - Family obligations affecting financial needs
- **Benefits** - Health insurance, 401(k) matching, pension
- **Other income sources** - Side hust

##10.AUDIT BUNDLE

###10.1.OVERVIEW



The final cell exports artifacts: run_manifest, graph_spec, and final_state. This step transforms the notebook from a transient demonstration into a reproducible system. Each run is documented with model identifier, timestamp, and graph topology.

The run_manifest records configuration context. The graph_spec serializes architecture. The final_state captures execution outcome. Together they form a minimal audit trail. Even though governance is not the focus of this pedagogical track, reproducibility remains foundational.

Exporting graph_spec is particularly instructive. It shows that architecture is data. The graph can be serialized, inspected, or even reloaded. Students should recognize that an agent system is not just code but a structured object that can be shared.

Saving final_state reinforces that outcomes are inspectable artifacts. In professional workflows, outputs must be stored, reviewed, and compared. This habit begins here.

This cell concludes the cycle by emphasizing durability. The architecture persists beyond the session. The system has identity. Students leave with a full mental model: configure, define state, define agents, build graph, visualize, execute, inspect, export. That cycle is the foundation upon which more advanced agentic architectures will be built.


###10.2.CODE AND IMPLEMENTATION

In [ ]:
# CELL 10 — Artifact Export (Run manifest + graph spec + final state)
artifacts = {
    "run_manifest": {
        "run_id": RUN_ID,
        "timestamp": TIMESTAMP,
        "model": MODEL
    },
    "graph_spec": graph.get_graph().to_json(),
    "final_state": result
}

with open("run_manifest.json", "w") as f:
    json.dump(artifacts["run_manifest"], f, indent=2)

with open("graph_spec.json", "w") as f:
    json.dump(artifacts["graph_spec"], f, indent=2)

with open("final_state.json", "w") as f:
    json.dump(result, f, indent=2)

print("Artifacts written:")
print(" - run_manifest.json")
print(" - graph_spec.json")
print(" - final_state.json")


##11.CONCLUSION

This first notebook marks the transition from thinking about large language models as conversational tools to thinking about them as components inside engineered systems. Its purpose was never to deliver financial advice. Its purpose was to demonstrate, in the most concrete and executable way possible, how an agentic workflow can be built as an explicit graph governed by state, routing, and termination rules. By the end of this notebook, the student should no longer see an “agent” as a personality or a chatbot, but as a modular transformation embedded within a larger structure.

The core achievement of this notebook lies in clarity. We defined a structured state object that carries the evolving knowledge of the system. We defined agents as role-specific transformations that read and write that state. We defined a graph that determines what happens next, not by conversational guesswork, but by explicit routing logic. Most importantly, we introduced a bounded retry loop, showing that intelligent systems must have limits. The presence of a retry counter and a termination path demonstrates that architecture is about control as much as it is about reasoning.

From a user perspective, this notebook accomplishes something subtle but powerful. It turns a messy, ambiguous financial input into a controlled triage process. Instead of attempting to answer prematurely, the system first evaluates completeness, asks for clarification when necessary, and only then proceeds to structure information. This mirrors how disciplined professionals operate. The student learns that responsible workflows are staged workflows. The model does not decide the sequence arbitrarily; the graph does.

Pedagogically, this notebook establishes the invariant principles that will govern the entire series. Every future example will still revolve around state, agents, and graphs. What will change is topology. We will introduce branching vetoes, parallel committees, tool integrations, retrieval routers, and supervisory constellations. But the mental model remains identical. Once a student understands how a conditional retry loop operates, more complex patterns become variations on a theme rather than entirely new concepts.

Another lasting lesson is that architecture is visible. The Mermaid diagram is not decoration. It is a representation of the system’s logic. Being able to read and explain that diagram is a core competency. In professional environments, clarity of architecture determines whether systems can be audited, extended, or trusted. This notebook begins cultivating that literacy.

Finally, the strict use of a fast, pinned model reinforces an essential truth: intelligence does not arise from model scale alone. It arises from structure. By keeping the cognitive engine stable and lightweight, we emphasized that the graph is the primary artifact. The student leaves this notebook understanding that scalable agentic systems are built deliberately, with defined roles and controlled transitions. That understanding is the foundation upon which all subsequent notebooks will expand.
